In [2]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

api_key = "f3898958-b76a-4a47-816f-0294f0c5103d"

BASE_URL = "https://api.hellodata.ai"

HEADERS = {
    "x-api-key": api_key
}

dimasset = pd.read_csv('data/DimAsset.csv')

In [3]:
def get_lat_lon(property):
    """Function to get latitude and longitude for a given property."""
    matches = dimasset[dimasset['AssetName'].str.contains(property, case=False, regex=False)].dropna(subset=['Latitude', 'Longitude'])

    if matches.empty:
        raise ValueError(f"No match found for property: {property}")

    lat, lon = matches[['Latitude', 'Longitude']].iloc[0]

    if pd.isna(lat) or pd.isna(lon):
        raise ValueError(f"Latitude or Longitude missing for property: {property}")

    return lat, lon

def fetch_property_data(property, lat=None, lon=None, zip_code=None):
    """Function to fetch property data using the provided latitude and longitude.
       If lat and lon are not provided, they are omitted from the query."""
    
    encoded_property_name = property
    
    # Create the querystring dictionary
    querystring = {"q": encoded_property_name}
    
    # Only add lat and lon if they are provided
    if lat is not None and lon is not None:
        querystring["lat"] = lat
        querystring["lon"] = lon
        querystring["max_distance"] = 0.1

    if zip_code is not None:
        querystring['zip_code'] = zip_code
    
    # Make the API request
    try:
        response = requests.get(f"{BASE_URL}/property/search", headers=HEADERS, params=querystring)
        response.raise_for_status()  # Raise HTTP errors (e.g., 404, 500)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"HTTP request error: {e}")
        return None
    except ValueError:
        print("Error parsing JSON response from property search.")
        return None

def fetch_property_details(property_id):
    """Function to fetch details for a specific property."""
    url = f"{BASE_URL}/property/{property_id}"

    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"HTTP request error while fetching property details: {e}")
        return None
    except ValueError:
        print("Error parsing JSON response from property details.")
        return None

def fetch_comparables(property_details):
    """Function to fetch comparables for a given property."""
    url = f"{BASE_URL}/property/comparables"
    payload = {"subject": property_details}

    try:
        response = requests.post(url, json=payload, headers=HEADERS)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"HTTP request error while fetching comparables: {e}")
        return None
    except ValueError:
        print("Error parsing JSON response from comparables.")
        return None

def get_comps(property):
    """Final function to get the comparables data for a property."""
    try:
        lat, lon = get_lat_lon(property)
    except ValueError as e:
        print(f"Error in get_lat_lon: {e}")
        return None

    # Fetch property data
    property_data = fetch_property_data(property, lat, lon)
    if not property_data or not isinstance(property_data, list):
        print(f"Unexpected response format from property search: {property_data}")
        return None

    try:
        property_id = property_data[0].get("id")
        if not property_id:
            raise KeyError("Missing 'id' in property response.")
    except (IndexError, KeyError) as e:
        print(f"Error extracting property ID: {e}")
        return None

    # Fetch property details
    property_details = fetch_property_details(property_id)
    if not property_details:
        return None

    # Fetch comparables
    response_data = fetch_comparables(property_details)
    if not response_data or 'comparables' not in response_data or not isinstance(response_data['comparables'], list):
        print(f"Unexpected response format for comparables: {response_data}")
        return None

    try:
        comps = pd.json_normalize(response_data['comparables'])
    except Exception as e:
        print(f"Error normalizing comparables data: {e}")
        return None

    return comps


In [4]:
def get_unit_history(property_details):
    try:
        if not isinstance(property_details, dict):
            raise TypeError(f"Expected dictionary for property_details, got {type(property_details)}")

        history_df = pd.DataFrame()

        building_name = property_details.get('building_name')
        availability = property_details.get('building_availability', [])
        num_units = property_details.get('number_units', 0)

        for unit_id, cur_availability in enumerate(availability):
            if not isinstance(cur_availability, dict):
                print(f"Skipping invalid unit data at index {unit_id}: {type(cur_availability)}")
                continue

            try:
                unit_name = cur_availability.get('unit_name')
                unit_group = f"{cur_availability.get('bed', 0)}x{cur_availability.get('bath', 0)}"

                half_baths = cur_availability.get('partial_bath', 0)
                if half_baths == 1:
                    unit_group += ".5"

                sqft = cur_availability.get('sqft')

                for pricing_id, cur_history in enumerate(cur_availability.get('history', [])):
                    if not isinstance(cur_history, dict):
                        print(f"Skipping invalid history data at index {pricing_id}: {type(cur_history)}")
                        continue

                    try:
                        effective_price = cur_history.get('effective_price')
                        from_date = cur_history.get('from_date')
                        to_date = cur_history.get('to_date')

                        cur_history_df = pd.DataFrame(
                            {"building_name": building_name,
                             "unit_name": unit_name,
                             "unit_group": unit_group,
                             "sqft": sqft,
                             "effective_price": effective_price,
                             "from_date": from_date,
                             "to_date": to_date}, index=[0]
                        )

                        history_df = pd.concat([history_df, cur_history_df])

                    except Exception as e:
                        print(f"Error processing history at index {pricing_id}: {e}")

            except Exception as e:
                print(f"Error processing unit at index {unit_id}: {e}")

        # Convert dates and handle invalid dates
        history_df["from_date"] = pd.to_datetime(history_df["from_date"], errors='coerce')
        history_df["to_date"] = pd.to_datetime(history_df["to_date"], errors='coerce')

        # Sort and calculate leased rate
        history_df.sort_values(by=["unit_name", "from_date"], inplace=True)
        history_df["next_from_date"] = history_df.groupby("unit_name")["from_date"].shift(-1)
        history_df["leased_rate"] = (history_df["to_date"] + pd.Timedelta(days=1) < history_df["next_from_date"]) | (history_df['next_from_date'].isna())

        # Expand rows for each date in range
        expanded_history = []
        for _, row in history_df.iterrows():
            try:
                date_range = pd.date_range(row["from_date"], row["to_date"])
                for single_date in date_range:
                    expanded_history.append({
                        "building_name": row["building_name"],
                        "unit_name": row["unit_name"],
                        "unit_group": row["unit_group"],
                        "sqft": row['sqft'],
                        "effective_price": row["effective_price"],
                        "date": single_date.strftime("%m/%d/%Y"),
                        "leased_rate": row["leased_rate"] if single_date == row["to_date"] else False
                    })
            except Exception as e:
                print(f"Error expanding history for row {row['unit_name']}: {e}")

        # Create new DataFrame
        expanded_history_df = pd.DataFrame(expanded_history)

        return expanded_history_df, num_units
    
    except Exception as e:
        print(f"Error processing property details: {e}")
        return pd.DataFrame(), 0


In [5]:
def get_net_leased(history_df, num_units):
    history_df = history_df.copy()
    history_df['date'] = pd.to_datetime(history_df['date'])
    
    building = history_df['building_name'].unique()[0]
    
    first_date = history_df['date'].min()
    last_date = history_df['date'].max()
    date_range = pd.date_range(first_date, last_date, freq='MS')

    net_leased_df = pd.DataFrame()
    
    for date in date_range:
        num_vacancies = len(history_df[history_df['date'] == date])

        vacancy_rate = num_vacancies / num_units

        net_leased_df = pd.concat([net_leased_df, pd.DataFrame({"property": building, "date": date, "net_leased": 1 - vacancy_rate}, index=[0])])

    
    return net_leased_df




In [6]:
def get_comp_net_leased(property):
    lat, lon = get_lat_lon(property=property)

    property_data = fetch_property_data(property=property, lat=lat, lon=lon)

    comps = get_comps(property=property)

    property_id = property_data[0].get("id")
    property_details = fetch_property_details(property_id)

    print(f"Getting history for {property}...")

    history_df, num_units = get_unit_history(property_details)

    net_leased_df = get_net_leased(history_df, num_units)

    for i in range(len(comps)):
        building_name = comps['building_name'][i]
        zip_code = comps['zip_code'][i]

        print(f"Getting net leased for {building_name}...")

        property_data = fetch_property_data(property=building_name, zip_code=zip_code)

        property_id = property_data[0].get("id")

        property_details = fetch_property_details(property_id=property_id)

        history_df, num_units = get_unit_history(property_details=property_details)

        net_leased_df = pd.concat([net_leased_df, get_net_leased(history_df=history_df, num_units=num_units)])

    net_leased_df = net_leased_df[net_leased_df['date'] >= '2024-01-01']

    net_leased_df['year_month'] = net_leased_df['date'].dt.to_period('M')

    net_leased_df.to_csv(f"data/{property} Comps Net Leased.csv")

In [7]:
def get_rolling_rates(unit_history, building_name):
    signed_leases = unit_history[unit_history['leased_rate'] == True]

    # Convert 'date' to datetime and extract year and month
    signed_leases['date'] = pd.to_datetime(signed_leases['date'])
    signed_leases['year_month'] = signed_leases['date'].dt.to_period('M')

    rolling_rates = pd.DataFrame()

    for i in sorted(signed_leases['year_month'].unique()):

        applicable_leases = (
            signed_leases[
                (signed_leases['year_month'] <= i) &
                (signed_leases['date'] >= '2024-01-01')
            ]
            .dropna(subset=['effective_price'])
            .groupby("unit_name", as_index=False)
            .last()
        )

        cur_rolling_rates = pd.DataFrame()  # Initialize as empty DataFrame

        if not applicable_leases.empty:
            sqft_sum = applicable_leases['sqft'].sum()
            avg_rent_per_sqft = applicable_leases['effective_price'].sum() / sqft_sum if sqft_sum > 0 else None

            cur_rolling_rates = pd.DataFrame({
                'building_name': [building_name],
                'year_month': [i],
                'avg_rent_roll': [applicable_leases['effective_price'].mean()],
                'avg_rent_per_sqft': [avg_rent_per_sqft]
            })

        if not cur_rolling_rates.empty:
            rolling_rates = pd.concat([rolling_rates, cur_rolling_rates], ignore_index=True)

    rolling_rates['building_name'] = building_name

    rolling_rates = rolling_rates[['building_name', 'year_month', 'avg_rent_roll', 'avg_rent_per_sqft']]

    return rolling_rates.sort_values(by="year_month", ascending=True).reset_index(drop=True)

In [8]:
def get_comp_rolling_rates(property):
    comps = get_comps(property)  # Fetch comps for the reference property

    rolling_rates_df = pd.DataFrame()  # Initialize an empty DataFrame for rate data

    # Iterate through each comp property and fetch their rate data
    for i in range(len(comps)):
        building_name = comps['building_name'][i]
        zip_code = comps['zip_code'][i]

        print(f"Getting rates for {building_name}...")

        try:
            # Fetch property data with latitude and longitude
            property_data = fetch_property_data(building_name, zip_code=zip_code)
            
            # If fetch_property_data fails, property_data will be None
            if property_data is None:
                print(f"Failed to fetch property data for {building_name}")
                continue  # Skip to the next property

            # Extract property ID
            property_id = property_data[0].get("id")

            # Fetch property details
            property_details = fetch_property_details(property_id)

            unit_history, num_units = get_unit_history(property_details)

            cur_rolling_rates = get_rolling_rates(unit_history, building_name)

            rolling_rates_df = pd.concat([rolling_rates_df, cur_rolling_rates])

        except Exception as e:
            print(f"Error getting rates for {building_name}: {e}")
            continue  # Skip to the next property in case of error

    # Fetch availability for the reference building
    building_name = property
    lat, lon = get_lat_lon(property)

    print(f"Getting rates for {building_name}...\n")

    try:
        # Fetch property data for the reference property with lat, lon
        property_data = fetch_property_data(building_name, lat=lat, lon=lon)

        if property_data is None:
            print(f"Failed to fetch property data for {building_name}")
            return rolling_rates_df  # Return the current DataFrame even if the fetch failed

        property_id = property_data[0].get("id")
        property_details = fetch_property_details(property_id)

        unit_history, num_units = get_unit_history(property_details)

        cur_rolling_rates = get_rolling_rates(unit_history, building_name)

        rolling_rates_df = pd.concat([rolling_rates_df, cur_rolling_rates])

    except Exception as e:
        print(f"Error getting details for reference property {building_name}: {e}")
        print(f"Lat: {lat}, Lon: {lon}\n")

    rolling_rates_df.to_csv(f"data/{property} Comps Rolling Rates.csv")

In [9]:
def predict_unit_mix(history_df):

    unit_mix = history_df.groupby('unit_group').agg(
        average_sqft=('sqft', 'mean'),
        count=('unit_name', 'nunique'),
    ).reset_index()

    unit_mix['prop'] = unit_mix['count'] / sum(unit_mix['count'])

    return(unit_mix)

In [10]:
property = "Cortland Rosslyn"

get_comp_rolling_rates(property)
get_comp_net_leased(property)

KeyboardInterrupt: 

In [ ]:
rolling_rates = pd.read_csv(f'data/{property} Comps Rolling Rates.csv')
net_leased = pd.read_csv(f'data/{property} Comps Net Leased.csv')

metrics = pd.merge(left=rolling_rates, right=net_leased, left_on=['building_name', 'year_month'], right_on=['property', 'year_month'])
metrics = metrics[['building_name', 'year_month', 'avg_rent_per_sqft', 'net_leased']]

metrics['rev_pasf'] = metrics['avg_rent_per_sqft'] * metrics['net_leased']

metrics['rev_pasf_rank'] = metrics.groupby('year_month')['rev_pasf'].rank(method='dense', ascending=False)

metrics['t1_rev_pasf_rank'] = metrics.groupby(['building_name'])['rev_pasf_rank'].shift(1)

metrics['performance_score'] = metrics.apply(
    lambda x: (x['t1_rev_pasf_rank'] - x['rev_pasf_rank']), 
    axis=1
)

metrics.to_csv(f'data/{property} Comp Metrics.csv')

In [11]:
id = fetch_property_data("Evergreen Lenox Park")[0]['id']
details = fetch_property_details(id)
history_df, num_units = get_unit_history(details)

display(history_df)

,building_name,unit_name,unit_group,sqft,effective_price,date,leased_rate
0,Evergreen Lenox Park,1101,2x2,1223,1745,09/13/2024,False
1,Evergreen Lenox Park,1101,2x2,1223,1745,09/14/2024,False
2,Evergreen Lenox Park,1101,2x2,1223,1745,09/15/2024,False
3,Evergreen Lenox Park,1101,2x2,1223,1745,09/16/2024,False
4,Evergreen Lenox Park,1101,2x2,1223,1745,09/17/2024,False
...,...,...,...,...,...,...,...
14556,Evergreen Lenox Park,6140,2x2,1134,1844,09/03/2023,False
14557,Evergreen Lenox Park,6140,2x2,1134,1844,09/04/2023,False
14558,Evergreen Lenox Park,6140,2x2,1134,1844,09/05/2023,False
14559,Evergreen Lenox Park,6140,2x2,1134,1828,09/06/2023,False
